# Load dim_order_payments từ Silver lên Gold

## Mục tiêu
- Load dữ liệu từ Silver.dim_order_payments → Gold.dim_order_payments
- Không cần biến đổi dữ liệu
- Chỉ cần copy và load vào Gold database

## Quy trình 3 bước
1. **Load from Silver** - Đọc dữ liệu từ Silver database
2. **Schema Definition** - Định nghĩa schema cho MySQL
3. **Load to Gold** - Load vào Gold database


## Bước 1: Import và Setup


In [1]:
# Import các thư viện cần thiết
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime

# Load biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối database
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_SILVER = os.getenv("DB_SILVER")
DB_GOLD = os.getenv("DB_GOLD")

# Tạo kết nối tới Silver và Gold database
silver_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_SILVER}")
gold_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_GOLD}")

print("Đã kết nối Silver và Gold database thành công")
print(f"Silver DB: {DB_SILVER}")
print(f"Gold DB: {DB_GOLD}")
print(f"Thời gian bắt đầu: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Đã kết nối Silver và Gold database thành công
Silver DB: winner_silver
Gold DB: winner_gold
Thời gian bắt đầu: 2025-10-17 23:30:18


## Bước 2: Load dữ liệu từ Silver


In [2]:
# Load dữ liệu dim_order_payments từ Silver database
print("=== LOADING DIM_ORDER_PAYMENTS FROM SILVER ===")

# Load toàn bộ bảng dim_order_payments
payments_df = pd.read_sql("SELECT * FROM dim_order_payments", silver_engine)

# Hiển thị thông tin cơ bản về dataset
print(f"Shape: {payments_df.shape}")
print(f"Columns: {list(payments_df.columns)}")
print(f"Memory usage: {payments_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Hiển thị sample data
print("\n=== SAMPLE DATA ===")
print(payments_df.head(5))

# Hiển thị data types
print("\n=== DATA TYPES ===")
print(payments_df.dtypes)

# Kiểm tra null values
print(f"\n=== NULL VALUES ===")
null_counts = payments_df.isnull().sum()
print(null_counts[null_counts > 0])


=== LOADING DIM_ORDER_PAYMENTS FROM SILVER ===
Shape: (40236, 8)
Columns: ['payment_id', 'payment_method', 'payment_amount', 'cod', 'cash', 'charged_by_card', 'charged_by_momo', 'charged_by_qrpay']
Memory usage: 14.30 MB

=== SAMPLE DATA ===
  payment_id payment_method  payment_amount cod cash charged_by_card  \
0  PAY_40616        Unknown             0.0   0    0               0   
1  PAY_40615        Unknown             0.0   0    0               0   
2  PAY_40614        Unknown             0.0   0    0               0   
3  PAY_40613        Unknown             0.0   0    0               0   
4  PAY_40612        Unknown             0.0   0    0               0   

  charged_by_momo charged_by_qrpay  
0               0                0  
1               0                0  
2               0                0  
3               0                0  
4               0                0  

=== DATA TYPES ===
payment_id           object
payment_method       object
payment_amount      float64

## Bước 3: Load vào Gold Database


In [3]:
# Load vào Gold Database
print("=== LOADING TO GOLD DATABASE ===")

from sqlalchemy.types import BigInteger, Integer, String, Text, DateTime, Float

# Định nghĩa schema mapping cho MySQL
dtype_mapping = {}

# ID fields - String (để xử lý mixed data types)
id_fields = ["payment_id", "order_id"]
for field in id_fields:
    if field in payments_df.columns:
        dtype_mapping[field] = String(100)

# String fields - VARCHAR(255)
string_fields = ["payment_method", "payment_status", "payment_gateway"]
for field in string_fields:
    if field in payments_df.columns:
        dtype_mapping[field] = String(255)

# Text fields - Text()
text_fields = ["payment_note", "transaction_id"]
for field in text_fields:
    if field in payments_df.columns:
        dtype_mapping[field] = Text()

# Numeric fields
float_fields = ["amount", "fee", "discount_amount"]
for field in float_fields:
    if field in payments_df.columns:
        dtype_mapping[field] = Float()

# DateTime fields
datetime_fields = ["created_at", "updated_at", "paid_at"]
for field in datetime_fields:
    if field in payments_df.columns:
        dtype_mapping[field] = DateTime()

print(f"Schema mapping defined for {len(dtype_mapping)} columns")

# Load vào Gold database
table_name = "dim_order_payments"

try:
    payments_df.to_sql(
        table_name,
        con=gold_engine,
        if_exists="replace",  # Ghi đè dữ liệu cũ
        index=False,
        dtype=dtype_mapping
    )
    
    print(f"Đã load {payments_df.shape[0]} records vào Gold: {table_name}")
    
    # Verify load
    verification_query = f"SELECT COUNT(*) as count FROM {table_name}"
    verification_result = pd.read_sql(verification_query, gold_engine)
    loaded_count = verification_result['count'].iloc[0]
    
    print(f"Verification: {loaded_count} records trong database")
    print(f"Schema: {len(dtype_mapping)} columns với explicit typing")
    
    # Sample data từ database
    sample_query = f"SELECT * FROM {table_name} LIMIT 5"
    sample_db = pd.read_sql(sample_query, gold_engine)
    print(f"\nSample data từ Gold database:\n{sample_db}")
    
except Exception as e:
    print(f"Error loading to Gold: {str(e)}")
    raise

print(f"\nLoad dim_order_payments completed successfully!")
print(f"Ready for EDA and analytics")


=== LOADING TO GOLD DATABASE ===
Schema mapping defined for 2 columns
Đã load 40236 records vào Gold: dim_order_payments
Verification: 40236 records trong database
Schema: 2 columns với explicit typing

Sample data từ Gold database:
  payment_id payment_method  payment_amount cod cash charged_by_card  \
0  PAY_40616        Unknown             0.0   0    0               0   
1  PAY_40615        Unknown             0.0   0    0               0   
2  PAY_40614        Unknown             0.0   0    0               0   
3  PAY_40613        Unknown             0.0   0    0               0   
4  PAY_40612        Unknown             0.0   0    0               0   

  charged_by_momo charged_by_qrpay  
0               0                0  
1               0                0  
2               0                0  
3               0                0  
4               0                0  

Load dim_order_payments completed successfully!
Ready for EDA and analytics


## Bước 4: Tạo Data Dictionary cho Gold


In [4]:
# Tạo Data Dictionary cho Gold
print("=== CREATING GOLD DATA DICTIONARY ===")

def get_business_meaning_vietnamese(column_name):
    """Get business meaning for each column in Vietnamese"""
    business_meanings = {
        # Định danh & Cơ bản
        "payment_id": "Mã định danh thanh toán duy nhất (khóa chính)",
        "order_id": "Mã định danh đơn hàng (khóa ngoại đến fact_orders)",
        
        # Thông tin thanh toán
        "payment_method": "Phương thức thanh toán (COD, Bank Transfer, etc.)",
        "payment_status": "Trạng thái thanh toán (pending, paid, failed, etc.)",
        "payment_gateway": "Cổng thanh toán (VNPay, MoMo, etc.)",
        "transaction_id": "Mã giao dịch từ cổng thanh toán",
        "payment_note": "Ghi chú về thanh toán",
        
        # Số tiền
        "amount": "Số tiền thanh toán (VND)",
        "fee": "Phí giao dịch (VND)",
        "discount_amount": "Số tiền giảm giá (VND)",
        
        # Thời gian
        "created_at": "Thời gian tạo bản ghi thanh toán",
        "updated_at": "Thời gian cập nhật bản ghi cuối cùng",
        "paid_at": "Thời gian thanh toán thành công"
    }
    return business_meanings.get(column_name, "Chưa định nghĩa ý nghĩa business")

# Tạo Data Dictionary
dict_data = []
for col in payments_df.columns:
    col_info = {
        "table_name": "dim_order_payments",
        "column_name": col,
        "dtype": str(payments_df[col].dtype),
        "sql_type": str(dtype_mapping.get(col, "Not defined")),
        "null_count": payments_df[col].isnull().sum(),
        "null_pct": round(payments_df[col].isnull().mean() * 100, 2),
        "unique_count": payments_df[col].nunique(),
        "sample_values": str(payments_df[col].dropna().unique()[:3].tolist()),
        "business_meaning": get_business_meaning_vietnamese(col),
        "extraction_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    dict_data.append(col_info)

data_dictionary = pd.DataFrame(dict_data)

# Hiển thị Data Dictionary
print(f"Generated Data Dictionary for {len(data_dictionary)} columns")
print("\n=== DATA DICTIONARY ===")
print(data_dictionary)

# Lưu vào file Excel
excel_path = "Technical_Document/Gold_dictionary.xlsx"

try:
    from openpyxl import load_workbook
    
    # Kiểm tra file Excel có tồn tại không
    try:
        # Load workbook hiện tại
        wb = load_workbook(excel_path)
        
        # Lấy sheet đầu tiên
        ws = wb.active
        
        # Kiểm tra xem có dữ liệu cũ không
        if ws.max_row > 1:
            print(f"Found existing data in {excel_path}, appending new data...")
        else:
            print(f"File {excel_path} exists but is empty, adding header and data...")
            
    except FileNotFoundError:
        print(f"File {excel_path} not found, creating new file...")
        wb = None
    except Exception as e:
        print(f"Error loading {excel_path}: {str(e)}, creating new file...")
        wb = None
    
    if wb is None:
        # Tạo file mới với header
        data_dictionary.to_excel(excel_path, index=False, sheet_name='Gold_Dictionary')
        print(f"✅ Created new file: {excel_path}")
    else:
        # Append vào file hiện tại
        from openpyxl.utils.dataframe import dataframe_to_rows
        
        # Tìm dòng cuối cùng có dữ liệu
        last_row = ws.max_row
        
        # Thêm dữ liệu mới từ dòng tiếp theo
        for r in dataframe_to_rows(data_dictionary, index=False, header=False):
            last_row += 1
            for c_idx, value in enumerate(r, 1):
                ws.cell(row=last_row, column=c_idx, value=value)
        
        # Lưu file
        wb.save(excel_path)
        print(f"✅ Appended {len(data_dictionary)} rows to: {excel_path}")
    
except Exception as e:
    print(f"❌ Error appending to Data Dictionary: {str(e)}")
    # Fallback: tạo file mới
    try:
        data_dictionary.to_excel(excel_path, index=False)
        print(f"✅ Created new file as fallback: {excel_path}")
    except Exception as e2:
        print(f"❌ Error creating fallback file: {str(e2)}")

# Summary Report
print(f"\n=== GOLD DICTIONARY SUMMARY ===")
print(f"Table: dim_order_payments")
print(f"Total columns: {len(data_dictionary)}")
print(f"Data Dictionary: {excel_path}")
print(f"Created at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n=== SAMPLE DICTIONARY ENTRIES ===")
sample_dict = data_dictionary[['column_name', 'dtype', 'business_meaning']].head(5)
print(sample_dict)


=== CREATING GOLD DATA DICTIONARY ===
Generated Data Dictionary for 8 columns

=== DATA DICTIONARY ===
           table_name       column_name    dtype      sql_type  null_count  \
0  dim_order_payments        payment_id   object  VARCHAR(100)           0   
1  dim_order_payments    payment_method   object  VARCHAR(255)           0   
2  dim_order_payments    payment_amount  float64   Not defined           0   
3  dim_order_payments               cod   object   Not defined           0   
4  dim_order_payments              cash   object   Not defined           0   
5  dim_order_payments   charged_by_card   object   Not defined           0   
6  dim_order_payments   charged_by_momo   object   Not defined           0   
7  dim_order_payments  charged_by_qrpay   object   Not defined           0   

   null_pct  unique_count                            sample_values  \
0       0.0         40236  ['PAY_40616', 'PAY_40615', 'PAY_40614']   
1       0.0             2                       ['Unkn